In [ ]:
# Imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams['figure.figsize'] = (15.0, 8.0) # set default size of plots
plt.rcParams['figure.facecolor'] = 'white'

pd.set_option('display.max_rows', None)

matplotlib.rcParams.update({'font.size': 15})

colors = ["#cccccc", "#FB9A99", "#B0DC89", "#33A02C", "#A6CEE3", "#1F78B4", "#FDBF6F", "#FF7F00"]

In [ ]:
def read_sym_cipher(filename):
    column_names = ["sym", "Algorithm", "HW/SW", "bytes_total", "seconds_total", "MB/s", "unused"]
    data = pd.read_csv(filename, names=column_names, on_bad_lines='skip')
    data = data.loc[data["sym"] == "sym"].reset_index(drop=True)
    data["Algorithm"] = data["Algorithm"].str.replace('_', '-', regex=True)
    data["Algorithm"] = data["Algorithm"].str.replace('-enc', '', regex=True)
    data["throughput"] = (data["bytes_total"].astype(int) / data["seconds_total"].astype(float)) / 1024 / 1024
    data = data[data["Algorithm"].str.contains("AAD") == False]
    data = data[data["Algorithm"].str.contains("GMAC") == False]
    data = data[data["Algorithm"].str.contains("-dec") == False]
    #data["Algorithm"] = data["Algorithm"].str.replace('GMAC Default', 'GMAC', regex=True)
    #grouped_by_database_type = filtered_by_database_type.groupby('database_type', as_index=False).median()
    data = data.drop('sym', axis=1).drop('HW/SW', axis=1).drop('bytes_total', axis=1).drop('seconds_total', axis=1) \
               .drop('MB/s', axis=1).drop('unused', axis=1)
    return data

def group_sym(data):
    data = data.groupby(["Algorithm"], as_index=False).agg([np.mean, np.std]).droplevel(axis=1, level=0).reset_index()
    data.columns = ["experiment", "mean", "std"]
    return data

def group_asym(data):
    data = data.groupby(["name"], as_index=False).agg([np.mean, np.std]).droplevel(axis=1, level=0).reset_index()
    data.columns = ["experiment", "mean", "std"]
    return data

def read_asym_cipher(filename):
    column_names = ["asym", "Algorithm", "key size", "operation", "avg ms", "ops/sec", "ops", "secs"]
    data = pd.read_csv(filename, names=column_names, on_bad_lines='skip')
    data = data.loc[data["asym"] == "asym"].reset_index(drop=True)
    data["name"] = data["Algorithm"] + " (" + data["operation"] + ")"
    data["name"] = data["name"].str.replace('\[ *SECP256R1\]', '', regex=True)
    data["name"] = data["name"].str.replace('   ', ' ', regex=True)
    data["name"] = data["name"].str.replace('  ', ' ', regex=True)
    data["name"] = data["name"].str.replace('_', '-', regex=True)
    data["name"] = data["name"].str.replace(' \(', '-', regex=True)
    data["name"] = data["name"].str.replace('\)', '', regex=True)
    data["name"] = data["name"].str.replace('key gen', 'keygen', regex=True)
    data["us"] = data["avg ms"].astype(float) * 1024
    #grouped_by_database_type = filtered_by_database_type.groupby('database_type', as_index=False).median()
    return data.drop('asym', axis=1).drop('Algorithm', axis=1).drop('key size', axis=1).drop('operation', axis=1) \
               .drop('avg ms', axis=1).drop('ops/sec', axis=1).drop('ops', axis=1).drop('secs', axis=1)

In [ ]:
native_raw_data_sym = read_sym_cipher('benchmark-native-large-pages.csv')
native_raw_data_asym = read_asym_cipher('benchmark-native-large-pages.csv')

native_sgx_raw_data_sym = read_sym_cipher('benchmark-native-sgx.csv')
native_sgx_raw_data_asym = read_asym_cipher('benchmark-native-sgx.csv')

wasm_raw_data_sym = read_sym_cipher('benchmark-wasm.csv')
wasm_raw_data_asym = read_asym_cipher('benchmark-wasm.csv')

wasm_sgx_raw_data_sym = read_sym_cipher('benchmark-wasm-sgx.csv')
wasm_sgx_raw_data_asym = read_asym_cipher('benchmark-wasm-sgx.csv')

In [ ]:
# Normalize the results based on native implementation
assert native_sgx_raw_data_sym.shape[0] == native_raw_data_sym.shape[0]
native_sgx_raw_data_sym["throughput"] = 1 / (native_sgx_raw_data_sym["throughput"] / native_raw_data_sym["throughput"])
assert native_sgx_raw_data_asym.shape[0] == native_raw_data_asym.shape[0]
native_sgx_raw_data_asym["us"] = native_sgx_raw_data_asym["us"] / native_raw_data_asym["us"]

assert wasm_raw_data_sym.shape[0] == native_raw_data_sym.shape[0]
wasm_raw_data_sym["throughput"] = 1 / (wasm_raw_data_sym["throughput"] / native_raw_data_sym["throughput"])
assert wasm_raw_data_asym.shape[0] == native_raw_data_asym.shape[0]
wasm_raw_data_asym["us"] = wasm_raw_data_asym["us"] / native_raw_data_asym["us"]

assert wasm_sgx_raw_data_sym.shape[0] == native_raw_data_sym.shape[0]
wasm_sgx_raw_data_sym["throughput"] = 1 / (wasm_sgx_raw_data_sym["throughput"] / native_raw_data_sym["throughput"])
assert wasm_sgx_raw_data_asym.shape[0] == native_raw_data_asym.shape[0]
wasm_sgx_raw_data_asym["us"] = wasm_sgx_raw_data_asym["us"] / native_raw_data_asym["us"]

In [ ]:
#native_sgx_raw_data_sym

In [ ]:
native_sgx_data_sym_grouped = group_sym(native_sgx_raw_data_sym)
wasm_data_sym_grouped = group_sym(wasm_raw_data_sym)
wasm_sgx_data_sym_grouped = group_sym(wasm_sgx_raw_data_sym)

wasm_sgx_data_sym_grouped.sort_values(by=["mean"], ascending=True)

In [ ]:
native_sgx_data_asym_grouped = group_asym(native_sgx_raw_data_asym)
wasm_data_asym_grouped = group_asym(wasm_raw_data_asym)
wasm_sgx_data_asym_grouped = group_asym(wasm_sgx_raw_data_asym)

native_sgx_data_asym_grouped

In [ ]:
# Retrieve the name of the experiment, order by TEE normalized run time
wasm_sgx_data_sym_indexes = wasm_sgx_data_sym_grouped.sort_values(by=["mean"], ascending=True).index
wasm_sgx_data_asym_indexes = wasm_sgx_data_asym_grouped.sort_values(by=["mean"], ascending=True).index

# Order the entries based on the TEE normalized run time
native_sgx_data_sym_export = native_sgx_data_sym_grouped.reindex(wasm_sgx_data_sym_indexes)
wasm_data_sym_export = wasm_data_sym_grouped.reindex(wasm_sgx_data_sym_indexes)
wasm_sgx_data_sym_export = wasm_sgx_data_sym_grouped.reindex(wasm_sgx_data_sym_indexes)

native_sgx_data_asym_export = native_sgx_data_asym_grouped.reindex(wasm_sgx_data_asym_indexes)
wasm_data_asym_export = wasm_data_asym_grouped.reindex(wasm_sgx_data_asym_indexes)
wasm_sgx_data_asym_export = wasm_sgx_data_asym_grouped.reindex(wasm_sgx_data_asym_indexes)

# Display the ordered experiments' name
display(",".join(wasm_sgx_data_sym_export["experiment"].values))
display(",".join(wasm_sgx_data_asym_export["experiment"].values))

In [ ]:
#
# Export script
#
def export_to_file(dataset, filename):
    dataset.to_csv(filename, index=False)
    
export_to_file(native_sgx_data_sym_export, 'sym_native_sgx_data_formatted.csv')
export_to_file(wasm_data_sym_export, 'sym_wasm_data_formatted.csv')
export_to_file(wasm_sgx_data_sym_export, 'sym_wasm_sgx_data_formatted.csv')
export_to_file(native_sgx_data_asym_export, 'asym_native_sgx_data_formatted.csv')
export_to_file(wasm_data_asym_export, 'asym_wasm_data_formatted.csv')
export_to_file(wasm_sgx_data_asym_export, 'asym_wasm_sgx_data_formatted.csv')

In [ ]:
#
## Stats for the paper.
#

def r_ratio(value, number_of_digits = 1):
    return f"{round(value, number_of_digits)}"

symmetric_ciphers_names = ["3DES","SHAKE256","SHAKE128","Camellia","AES-192-CBC","AES-256-CBC","AES-CCM","AES-128-CBC","AES-256-GCM","AES-192-GCM","AES-128-GCM","CHACHA"]
crypto_sym_ratio_wasm_to_native = r_ratio(wasm_data_sym_export[wasm_data_sym_export["experiment"].isin(symmetric_ciphers_names)]["mean"].agg(np.mean))
crypto_sym_ratio_wasm_sgx_to_native = r_ratio(wasm_sgx_data_sym_export[wasm_sgx_data_sym_export["experiment"].isin(symmetric_ciphers_names)]["mean"].agg(np.mean))

hash_ciphers_names = ["SHA3-512","SHA3-384","SHA3-256","SHA3-224""MD5","HMAC-MD5","HMAC-SHA224","HMAC-SHA256","SHA","HMAC-SHA","SHA-224","SHA-256","PBKDF2","POLY1305","HMAC-SHA512","HMAC-SHA384","SHA-384","SHA-512"]
crypto_hash_ratio_wasm_to_native = r_ratio(wasm_data_sym_export[wasm_data_sym_export["experiment"].isin(hash_ciphers_names)]["mean"].agg(np.mean))
crypto_hash_ratio_wasm_sgx_to_native = r_ratio(wasm_sgx_data_sym_export[wasm_sgx_data_sym_export["experiment"].isin(hash_ciphers_names)]["mean"].agg(np.mean))

asymmetric_ciphers_names = ["ECDHE-agree","ECDSA-sign","ECDSA-verify","ECC-keygen","RSA-private","RSA-public","DH-agree","DH-keygen"]
crypto_asym_ratio_wasm_to_native = r_ratio(wasm_data_asym_export[wasm_data_asym_export["experiment"].isin(asymmetric_ciphers_names)]["mean"].agg(np.mean))
crypto_asym_ratio_wasm_sgx_to_native = r_ratio(wasm_sgx_data_asym_export[wasm_sgx_data_asym_export["experiment"].isin(asymmetric_ciphers_names)]["mean"].agg(np.mean))

In [ ]:
#
## Export to LaTeX
#

f = open("crypto-export.tex", "w")

f.write(f"\\def\\cryptoSymRatioWasmToNative{{{crypto_sym_ratio_wasm_to_native}}}\n")
f.write(f"\\def\\cryptoSymRatioWasmSgxToNative{{{crypto_sym_ratio_wasm_sgx_to_native}}}\n")
f.write(f"\\def\\cryptoHashRatioWasmToNative{{{crypto_hash_ratio_wasm_to_native}}}\n")
f.write(f"\\def\\cryptoHashRatioWasmSgxToNative{{{crypto_hash_ratio_wasm_sgx_to_native}}}\n")
f.write(f"\\def\\cryptoAsymRatioWasmToNative{{{crypto_asym_ratio_wasm_to_native}}}\n")
f.write(f"\\def\\cryptoAsymRatioWasmSgxToNative{{{crypto_asym_ratio_wasm_sgx_to_native}}}\n")

f.close()